## Simulation Update

이전 파일에 비해 아래 항목들이 추가가 되었습니다.
- OSRM을 사용하여 차량 경로를 뽑아올 때, 교통수단을 선택할 수 있도록 했습니다. 차량의 경우 `car`, 도보의 경우 `foot`을 입력하면 됩니다.
- 

##### Import

In [1]:
import numpy as np
import itertools
import requests
import polyline
import json
import os
import math

import random as rd
import pandas as pd

from datetime import datetime, timedelta

from shapely.geometry import Point

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import warnings 

warnings.filterwarnings('ignore')

##### 좌표 거리 생성 함수

In [2]:
# 직선 거리 계산 함수
def calculate_straight_distance(lat1, lon1, lat2, lon2):
# 직선 거리 계산
# 입력값: 출발지 위도, 경도 / 도착지 위도, 경도
    # 지구 반경 (킬로미터 단위)
    km_constant = 3959* 1.609344
    
    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    
    # 위도 및 경도 차이 계산
    dlat = lat2 - lat1 # 도착지 위도 - 출발지 위도
    dlon = lon2 - lon1 # 도착지 경도 - 출발지 경도

    # Haversine 공식 계산
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    # 거리 계산 (킬로미터 단위)
    km = km_constant * c
    
    return km # 출발지-도착지 사이의 직선 거리 반환

##### trips 데이터 생성 함수

In [ ]:
#### osrm 페키지로 경로 추출
# 입력으로 받은 출발지와 목적지 좌표를 이용하여 경로 정보를 가져오는 함수
def get_res(point, mode = 'foot'): # 도보 이용
   # point: 출발지와 목적지 좌표 정보를 포함하는 리스트[lat1, lon1, lat2, lon2]
   # mode: 이동 수단 (기본값: 도보)

   status = 'defined'

   # 요청을 재시도할 수 있도록 세션 객체 생성 및 설정
   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   #### url 생성 코드
   # 전체 경로 정보를 요청
   overview = '?overview=full'
   # lon, lat, lon, lat 형식의 출발지 목적지 좌표
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}"
   # 보행경로 url
   url = f'http://router.project-osrm.org/route/v1/{mode}/'
   # 경로 정보 요청
   r = session.get(url + loc + overview) 
   
   # 만약 경로가 안뜰 때 대체 결과 생성
   if r.status_code!= 200:
      
      status = 'undefined'
      
       # 직선 거리 계산
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # 경로 정보 생성 (출발지와 목적지 좌표만 포함)
      route = [[point[0], point[1]], [point[2], point[3]]]

      # 소요 시간 및 타임스탬프 계산 (가정: 보행 속도 10km/h)
      speed_km = 10#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   # 경로 정보를 성공적으로 가져온 경우, JSON 응답을 반환
   res = r.json()   
   return res, status

In [4]:
# 경로를 가는데 걸리는 시간과 거리 추출 함수
def extract_duration_distance(res, speed_kmh):
   # get_res함수에서 추출된 데이터에서 시간과 거리 뽑기
   # 입력값: res(get_res함수에서 추출된 데이터), 속도 (km/h)
   
   distance = res['routes'][0]['distance']
   # JSON 응답에서 첫 번째 경로의 거리 값을 추출
   # m 단위로 거리 추출
   # duration = res['routes'][0]['duration']/(60)  # 분 단위로 변환
   
   # 속도로 시간 계산
   speed_kmh = speed_kmh  # km/h
   speed_mps = speed_kmh * 1000 / 3600  # 속도를 m/s로 변환
   duration = distance / speed_mps / 60  # 분 단위로 변환
   
   return duration, distance # 소요 시간, 거리 반환

# 경로 추출 함수
def extract_route(res):
    # 입력값: res(get_res함수에서 추출된 데이터)
   
    # get_res함수에서 추출된 데이터에서 경로 뽑기
    # 경로가 인코딩 되어 있기 때문에 아래 함수를 써서 디코딩해주어야지 위경도로 이루어진 경로가 나옴
    route = polyline.decode(res['routes'][0]['geometry'])
    
    # 사용할 형식에 맞춰 위경도 좌표의 위치를 바꿔주는 것
    route = list(map(lambda data: [data[1],data[0]] ,route))
    # data: [위도, 경도] 형식의 좌표 쌍 -> [경도, 위도] 형식의 좌표 쌍

    return route # [[127.0, 37.0], [127.1, 37.1], ...] 형식의 경로 반환

In [5]:
# 총 걸리는 시간을 경로의 거리 기준으로 쪼개주는 함수
def extract_timestamp(route, duration):
    # 입력값: route(경로 정보), duration(총 소요 시간)
    
    # 리스트를 numpy이 배열로 변경
    rt = np.array(route)
    # 리스트를 수평 기준으로 합치기
    rt = np.hstack([rt[:-1,:], rt[1:,:]])
    # [출발점_lat, 출발점_lon, 도착점_lat, 도착점_lon] 형식으로 변환

    # 각각 직선거리 추출(리스트 형태)
    per = calculate_straight_distance(rt[:,1], rt[:,0], rt[:,3], rt[:,2])
    # 출발점 경도, 출발점 위도, 도착점 경도, 도착점 위도를 이용하여 직선거리 계산

    # 각각의 직선거리를 전체 직선거리의 합으로 나누기
    per = per / np.sum(per)
    # 각 구간의 직선 거리 비율 계산
    # 전체 경로에서 해당 구간이 차지하는 비율 계산

    # 계산된 비율을 기반으로 각 지점 도착 예상 시간 계산
    timestamp = per * duration
    ## 각 구간의 비율에 전체 소요 시간을 곱해 각 구간의 소요 시간 계산
    
    # 각 구간의 소요 시간을 누적하여 더하기
    timestamp = np.hstack([np.array([0]),timestamp])

    # 타임스탬프 배열의 누적 합을 계산하여 각 지점의 예상 도착 시간을 생성
    timestamp = list(itertools.accumulate(timestamp)) 
    
    return timestamp # 각 지점의 예상 도착 시간 리스트 e.g. [0, 10, 20, 30, ...] 반환

In [6]:
from functools import partial

# 모든 함수를 한번에 실행하는 코드(trips 데이터의 형태로 저장)
def osrm_routing_machine(O, D, mode, speed_kmh):
   # 입력값: 출발지 좌표, 목적지 좌표, 이동 수단, 속도

   # osrm 데이터 생성
   osrm_base, status = get_res([O.x, O.y, D.x, D.y], mode)
   
   # osrm 데이터가 생성 됬으면 진행
   if status == 'defined':
      # 거리 및 걸리는 시간 추출
      duration, distance = extract_duration_distance(osrm_base, speed_kmh)
      # 경로 추출
      route = extract_route(osrm_base)
      # timestamp 생성
      timestamp = extract_timestamp(route, duration)
      # 결과 저장
      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
      
      return result
   else: 
      return osrm_base # 경로 데이터 없을 때는 직선거리 데이터 반환
   
# OD_data 한쌍일 때 osrm_routing_machine작동함수
def osrm_routing_machine_multiprocess(OD, mode, speed_kmh):
   O, D = OD
   result = osrm_routing_machine(O, D, mode, speed_kmh)
   return result
# OD_data 데이터가 리스트쌍 일때의 osrm_routing_machine 작동함수
def osrm_routing_machine_multiprocess_all(OD_data, mode, speed_kmh):
    results = list(map(partial(osrm_routing_machine_multiprocess, mode = mode, speed_kmh=speed_kmh), OD_data))
    return results

##### od 데이터 생성

In [7]:
# 랜덤한 쌍의 od 데이터(시작점과 도착점이 같이 않게 뜸)
def get_OD_data(point, num = 10) :
    OD_data = []

    # 10개의 랜덤쌍 좌표 생성
    for _ in range(num):
        # 포인트 좌표의 key값을 이용하여 랜덤 쌍 생성
        neighborhood1, neighborhood2 = rd.sample(list(point.keys()), 2)
        # 랜덤쌍의 첫번째 값을 시작점으로 두번째 값을 도착점으로 설정
        start_point = point[neighborhood1]
        end_point = point[neighborhood2]
        
        # 시작점과 끝점을 포인트 좌표로 변경 
        O = Point(start_point)
        D = Point(end_point)
        # 시작점과 출발점을 리스트로 만들어 리스트에 추가
        OD_data.append([O, D])
    
    return OD_data

# 초 단위 변환 함수
def convert_to_minutes(time):
    return time.hour * 60 + time.minute + time.second / 60

# 랜덤한 시간 생성
def generate_start_times(num_passengers, start_hour=9, end_hour=10):
    start_times = []
    for _ in range(num_passengers):
        # 랜덤한 출발 시간 (분 단위)
        start_time = datetime(2024, 1, 1, start_hour, 0, 0) + timedelta(
            minutes=rd.randint(0, (end_hour - start_hour) * 60 - 1, ), seconds=rd.randint(0, 59)
        )
        # 시간을 분 단위로 변환
        start_time_minutes = convert_to_minutes(start_time)
        start_times.append(start_time_minutes)
    return start_times

# 출발시간 기반으로 탑승시간 계산 함수
def calculate_boarding_time(start_times):
    boarding_times = []
    for start_time in start_times:
        # 버스는 10분 간격으로 출발한다고 가정
        boarding_time = math.ceil(start_time / 10) * 10
        boarding_times.append(boarding_time)
    return boarding_times

# OD 데이터와 출발 시간, 탑승 시간 데이터프레임 생성
def create_od_dataframe(point, num_passengers, start_hour=9, end_hour=10):
    # OD 데이터 생성
    OD_data = get_OD_data(point, num_passengers)
    # 랜덤 출발 시간 생성
    start_times = generate_start_times(num_passengers, start_hour, end_hour)
    # 탑승 시간 계산
    boarding_times = calculate_boarding_time(start_times)
    
    # 데이터프레임 생성
    data = []
    for (start_station, end_station), start_time, boarding_time in zip(OD_data, start_times, boarding_times):
        data.append({
            "출발시간": start_time,
            "탑승시간": boarding_time,
            "탑승위치(Station_id)": start_station,
            "하차위치(Station_id)": end_station,
        })
    df = pd.DataFrame(data)
    return df

def extract_od_and_start_time(df):
    # OD 데이터를 추출
    od_data = [[row['탑승위치(Station_id)'], row['하차위치(Station_id)']] for _, row in df.iterrows()]
    # 출발시간 데이터를 추출
    start_time = df['출발시간'].tolist()
    boarding_time = df['탑승시간'].tolist()
    return od_data, start_time, boarding_time

In [8]:
# 데이터 좌표
point = {
    "중앙시장사거리" : [127.131770, 37.440888],
    "숯골사거리" : [127.142398 , 37.444055],
    "동부센트레빌2단지아파트" : [127.129460 , 37.447540],
    "수진역" : [127.140851 , 37.437443],
    "개별용달" : [127.139292 , 37.446605],
    "버거킹" : [127.150505 , 37.442235],
}

In [9]:
df = create_od_dataframe(point, 40, start_hour=9, end_hour=10)
df.head()

,출발시간,탑승시간,탑승위치(Station_id),하차위치(Station_id)
0,595.350000,600,POINT (127.13177 37.440888),POINT (127.150505 37.442235)
1,567.383333,570,POINT (127.139292 37.446605),POINT (127.142398 37.444055)
2,583.383333,590,POINT (127.12946 37.44754),POINT (127.13177 37.440888)
3,558.983333,560,POINT (127.142398 37.444055),POINT (127.150505 37.442235)
4,563.783333,570,POINT (127.139292 37.446605),POINT (127.150505 37.442235)


In [10]:
# 사용할 수 있는 형태로 변경
OD_data, start_time, boarding_time = extract_od_and_start_time(df)

In [11]:
OD_data

[[<POINT (127.132 37.441)>, <POINT (127.151 37.442)>],
 [<POINT (127.139 37.447)>, <POINT (127.142 37.444)>],
 [<POINT (127.129 37.448)>, <POINT (127.132 37.441)>],
 [<POINT (127.142 37.444)>, <POINT (127.151 37.442)>],
 [<POINT (127.139 37.447)>, <POINT (127.151 37.442)>],
 [<POINT (127.132 37.441)>, <POINT (127.129 37.448)>],
 [<POINT (127.132 37.441)>, <POINT (127.141 37.437)>],
 [<POINT (127.142 37.444)>, <POINT (127.139 37.447)>],
 [<POINT (127.151 37.442)>, <POINT (127.142 37.444)>],
 [<POINT (127.142 37.444)>, <POINT (127.139 37.447)>],
 [<POINT (127.129 37.448)>, <POINT (127.139 37.447)>],
 [<POINT (127.139 37.447)>, <POINT (127.151 37.442)>],
 [<POINT (127.129 37.448)>, <POINT (127.151 37.442)>],
 [<POINT (127.151 37.442)>, <POINT (127.141 37.437)>],
 [<POINT (127.142 37.444)>, <POINT (127.132 37.441)>],
 [<POINT (127.141 37.437)>, <POINT (127.142 37.444)>],
 [<POINT (127.129 37.448)>, <POINT (127.139 37.447)>],
 [<POINT (127.141 37.437)>, <POINT (127.139 37.447)>],
 [<POINT (

In [12]:
start_time

[595.35,
 567.3833333333333,
 583.3833333333333,
 558.9833333333333,
 563.7833333333333,
 593.9666666666667,
 562.25,
 571.4333333333333,
 563.6166666666667,
 549.8666666666667,
 553.25,
 578.8333333333334,
 578.05,
 552.8166666666667,
 588.3666666666667,
 541.7833333333333,
 543.6166666666667,
 552.7166666666667,
 574.8666666666667,
 589.4166666666666,
 543.85,
 581.0333333333333,
 595.0666666666667,
 577.8,
 544.6,
 545.5666666666667,
 549.1666666666666,
 591.4166666666666,
 594.05,
 592.1833333333333,
 556.6833333333333,
 559.9833333333333,
 569.5,
 547.95,
 547.5166666666667,
 566.7333333333333,
 553.2666666666667,
 581.7666666666667,
 551.7333333333333,
 555.0]

In [13]:
boarding_time

[600,
 570,
 590,
 560,
 570,
 600,
 570,
 580,
 570,
 550,
 560,
 580,
 580,
 560,
 590,
 550,
 550,
 560,
 580,
 590,
 550,
 590,
 600,
 580,
 550,
 550,
 550,
 600,
 600,
 600,
 560,
 560,
 570,
 550,
 550,
 570,
 560,
 590,
 560,
 560]

##### trips 데이터 생성

- 뒤의 20명은 OSRM을 통해 목적지까지 라우팅하되, 통행속도를 5km/h로 가정

- 탑승시간이란 컬럼 없이, 출발시간에 바로 출발

In [14]:
## 뒤의 20명은 OSRM을 통해 목적지까지 라우팅하되, 통행속도를 5km/h로 가정.
## 여기선 탑승시간이란 컬럼 없이, 출발시간에 바로 출발할 수 있도록
# OD, DO 포인트에 대해서 각각의 trips데이터를 생성
OD_data_foot = OD_data[20:]
OD_results_foot = osrm_routing_machine_multiprocess_all(OD_data, 'foot', 5)

In [15]:
OD_data_foot # 출발지 - 도착지 좌표

[[<POINT (127.132 37.441)>, <POINT (127.142 37.444)>],
 [<POINT (127.142 37.444)>, <POINT (127.151 37.442)>],
 [<POINT (127.139 37.447)>, <POINT (127.151 37.442)>],
 [<POINT (127.141 37.437)>, <POINT (127.139 37.447)>],
 [<POINT (127.139 37.447)>, <POINT (127.141 37.437)>],
 [<POINT (127.142 37.444)>, <POINT (127.132 37.441)>],
 [<POINT (127.141 37.437)>, <POINT (127.139 37.447)>],
 [<POINT (127.139 37.447)>, <POINT (127.141 37.437)>],
 [<POINT (127.151 37.442)>, <POINT (127.141 37.437)>],
 [<POINT (127.142 37.444)>, <POINT (127.141 37.437)>],
 [<POINT (127.142 37.444)>, <POINT (127.129 37.448)>],
 [<POINT (127.129 37.448)>, <POINT (127.132 37.441)>],
 [<POINT (127.141 37.437)>, <POINT (127.142 37.444)>],
 [<POINT (127.141 37.437)>, <POINT (127.132 37.441)>],
 [<POINT (127.132 37.441)>, <POINT (127.142 37.444)>],
 [<POINT (127.129 37.448)>, <POINT (127.141 37.437)>],
 [<POINT (127.141 37.437)>, <POINT (127.142 37.444)>],
 [<POINT (127.139 37.447)>, <POINT (127.132 37.441)>],
 [<POINT (

In [16]:
OD_results_foot # 출발지 - 도착지 좌표에 대한 결과

[{'route': [[127.13177, 37.44088],
   [127.13179, 37.44089],
   [127.13182, 37.4409],
   [127.13359, 37.44141],
   [127.13522, 37.4419],
   [127.1356, 37.44202],
   [127.13616, 37.44217],
   [127.13705, 37.44244],
   [127.13725, 37.44249],
   [127.13784, 37.44268],
   [127.13806, 37.44275],
   [127.1389, 37.443],
   [127.13902, 37.44303],
   [127.1397, 37.44323],
   [127.13984, 37.44327],
   [127.14079, 37.44355],
   [127.14132, 37.44372],
   [127.14233, 37.44402],
   [127.14247, 37.44406],
   [127.1429, 37.44423],
   [127.14322, 37.44438],
   [127.14342, 37.44449],
   [127.14386, 37.44477],
   [127.14396, 37.44484],
   [127.14412, 37.44496],
   [127.14461, 37.44533],
   [127.14509, 37.44576],
   [127.14523, 37.44595],
   [127.14536, 37.44612],
   [127.14551, 37.44632],
   [127.146, 37.44595],
   [127.1463, 37.44573],
   [127.14802, 37.44479],
   [127.14812, 37.44473],
   [127.14967, 37.44395],
   [127.15004, 37.44375],
   [127.15074, 37.44339],
   [127.15051, 37.44317],
   [127.14993,

In [17]:
# 생성된 트립 데이터에 출발시간을 변경
def update_timestamps_with_start_time(OD_results, start_times):
    # OD_results와 start_times를 순회하며 타임스탬프 갱신
    updated_results = []
    for result, start_time in zip(OD_results, start_times):
        # 기존 timestamp를 start_time과 합산
        updated_timestamps = [t + start_time for t in result['timestamp']]
        # 기존 결과를 복사하고 timestamp를 업데이트
        updated_result = result.copy()
        updated_result['timestamp'] = updated_timestamps
        updated_results.append(updated_result)
    return updated_results

start_time_foot = start_time[20:]
updated_OD_results_foot = update_timestamps_with_start_time(OD_results_foot, start_time_foot)

In [ ]:
print(updated_OD_results_foot[0]['timestamp'][0])
print(updated_OD_results_foot[0]['timestamp'][-1])

570.5166666666667
582.5694666666667


- 앞의 20명은 OSRM을 통해 목적지까지 라우팅. 통행속도 30km/h로 주행
- 배차간격을 10분이라고 가정하고, 이에 맞춰서 탑승시간이라는 컬럼을 생성하는 코드 추가.
모든 정류소에서 10분마다 차가 출발한다고 가정.

In [ ]:
OD_data_car = OD_data[0:20]
OD_results_car = osrm_routing_machine_multiprocess_all(OD_data_car, 'car', 30)

KeyboardInterrupt: 

In [ ]:
# 생성된 트립 데이터에 출발시간을 변경
def update_timestamps_route(OD_results, start_times, boarding_time):
    # OD_results와 start_times를 순회하며 타임스탬프 갱신
    updated_results = []
    for result, start_time, boarding_time in zip(OD_results, start_times, boarding_time):
        # 기존 timestamp를 boarding_time으로 조정하고, start_time을 맨 앞에 추가
        updated_timestamps = [start_time] + [t + boarding_time for t in result['timestamp']]
        # 기존 결과를 복사하고 timestamp를 업데이트
        updated_result = result.copy()
        updated_result['timestamp'] = updated_timestamps
        
        # route의 첫 번째 항목 복제 후 맨 앞에 추가
        updated_result['route'].insert(0, updated_result['route'][0])
        
        updated_results.append(updated_result)
    return updated_results

start_time_car = start_time[0:20]
boarding_time_car = boarding_time[0:20]

updated_OD_results_car = update_timestamps_route(OD_results_car, start_time_car, boarding_time_car)

In [ ]:
print(updated_OD_results_car[0]['timestamp'][0])
print(updated_OD_results_car[0]['timestamp'][-1])

589.7833333333333
591.663


##### 포인트 데이터 생성 ( CAR를 타기 전 대기 )

In [ ]:
# 시뮬레이션에서 사람이 대기하다가 이동하는 것을 위해서 포인트 데이터 생성
# ScatterplotLayer에 필요한 데이터 생성 함수
def create_scatterplot_data(data):
    scatterplot_data = []
    for item in data:
        if "route" in item and "timestamp" in item:
            start_point = item["route"][0]  # 첫 번째 좌표
            start_time = item["timestamp"][0]  # 타임스탬프 시작
            end_time = item["timestamp"][1] if len(item["timestamp"]) > 1 else start_time  # 타임스탬프 종료
            scatterplot_data.append({
                "coordinates": start_point,
                "timestamp": [start_time, end_time],
            })
    return scatterplot_data

# 데이터 생성
scatterplot_data = create_scatterplot_data(updated_OD_results_car)

In [ ]:
scatterplot_data[0]

{'coordinates': [127.13931, 37.44657], 'timestamp': [589.7833333333333, 590.0]}

##### 정류장 위치 아이콘 데이터

In [ ]:
icon_data = [{"name": name, "coordinates": coordinates} for name, coordinates in point.items()]
icon_data

[{'name': '중앙시장사거리', 'coordinates': [127.13177, 37.440888]},
 {'name': '숯골사거리', 'coordinates': [127.142398, 37.444055]},
 {'name': '동부센트레빌2단지아파트', 'coordinates': [127.12946, 37.44754]},
 {'name': '수진역', 'coordinates': [127.140851, 37.437443]},
 {'name': '개별용달', 'coordinates': [127.139292, 37.446605]},
 {'name': '버거킹', 'coordinates': [127.150505, 37.442235]}]

##### 데이터에서 TIMESTAMP의 최소, 최대값 확인

In [ ]:
###### 최대 시간을 봐서 시뮬레이션의 min, max 시간에 활용( 안 짤리도록 )
# all_timestamps = []

# for item in updated_OD_results_car + updated_OD_results_foot:
#     all_timestamps.extend(item['timestamp'])

# 리스트 컴프리헨션 쓰면 아주 편합니다.
all_timestamps = [t for item in updated_OD_results_car + updated_OD_results_foot for t in item['timestamp']]
# 최대값 계산
max_timestamp = max(all_timestamps)
min_timestamp = min(all_timestamps)

min_timestamp, max_timestamp

(540.3, 626.9428)

##### 데이터 저장

In [ ]:
# 데이터 저장
path = '../simulation_modify/public/data/'

with open(os.path.join(path + 'trips_foot.json'), 'w', encoding='utf-8') as file:
    json.dump(updated_OD_results_foot, file)
    
with open(os.path.join(path + 'trips_car.json'), 'w', encoding='utf-8') as file:
    json.dump(updated_OD_results_car, file)
    
with open(os.path.join(path + 'icon_data.json'), 'w', encoding='utf-8') as file:
    json.dump(icon_data, file)
    
with open(os.path.join(path + 'trips_car_point.json'), 'w', encoding='utf-8') as file:
    json.dump(scatterplot_data, file)

#### DECKGL LAYER

##### [1] SCATTERPLOT LAYER

- DECKGL에서 점을 표시할 때 사용
```JS
    new ScatterplotLayer({
      id: 'scatterplot-layer',  // LAYER ID
      data: point_car, // DATA
      getPosition: d => d.coordinates, // 위치
      getFillColor: [255, 255, 255], // 색
      getRadius: d => 3, // 점의 반지름
      getLineWidth: 3, //선의 두께
      radiusScale: 2, //반지름 값의 스케일( 확대, 축소)
      pickable: true, // 상호작용 여부 ( 클릭, 마우스 오버 )
      opacity: 0.5, // 투명도
    }),


```

- 만약 시뮬레이션에서 ScatterplotLayer이 나타낫다가 사라지게 하고 싶다면 아래 코드를 사용
- ScatterplotLayer에 사용하는 데이터에도 TIMESTAMP가 있어야 함
- 입력 데이터에서 특정 시간 범위에 속하는 데이터만 포함된 배열을 반환하여 시각화 가능
```JS
    const currData = (data, time) => {
    // 필터링된 데이터를 저장할 배열
    const arr = [];

    // 데이터 배열을 순회하며 각 항목에 대해 처리
    data.forEach((v) => {
      const timestamp = v.timestamp; // 데이터의 타임스탬프 배열
      const s_t = timestamp[0]; // 타임스탬프 시작 시간
      const e_t = timestamp[timestamp.length - 1]; // 타임스탬프 종료 시간

      // 현재 시간(time)이 타임스탬프 범위(s_t, e_t) 내에 있는 경우
      if (s_t <= time && e_t >= time) {
        arr.push(v); // 해당 데이터를 결과 배열에 추가
      }
  });

  // 필터링된 데이터 배열 반환
  return arr;
};
```


##### [2] ICON LAYER

- DECKGL에서 아이콘( EX. MARKER ) 표시할 때 사용

```JS
    new IconLayer({
      id: "location", // LAYER ID
      data: stop, // DATA
      sizeScale: 7, // ICON 크기 스케일 개수
      iconAtlas: // ICON 주소
        "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/icon-atlas.png",
      iconMapping: ICON_MAPPING, // 스프라이트 시트 내 아이콘 위치 및 크키
      getIcon: d => "marker", // ICON 이름
      getSize: 2, // ICON 크기
      getPosition: d => d.coordinates, // 위치( 위경도 )
      getColor: [255, 0, 0], // 색
      opacity: 1, // 투명도
      mipmaps: false, // 텍스처 품질 및 메모리 최적화 사용 여부
      pickable: true, // ICON 상호작용 여부
      radiusMinPixels: 2, // 아이콘 최소 반지름
      radiusMaxPixels: 2, // 아이콘 최대 반지름
    }),


```

##### [3] POLYGON LAYER

- 3D 입체를 표시할 때 사용
```JS
      
    new PolygonLayer({
      id: 'buildings', // LAYER ID
      data: building, // DATA
      extruded: true, // 다각형을 3D로 할지의 여부 ( TRUE : 높이 값을 사용하여 렌더링 )
      wireframe: false, //다각형의 와이어 프레임 ( TRUE : 선만 보이게 렌더링 )
      opacity: 0.5, // 투명도
      getPolygon: f => f.coordinates, // 위경도
      getElevation: f => f.height, // 높이
      getFillColor: DEFAULT_THEME.buildingColor, // 색
      material: DEFAULT_THEME.material // 다각형의 재질
    }),

```

##### [4] LINE, PATH LAYER

- 선을 그을 때 사용
- LINE LAYER는 NODE( 점 데이터 ) 사용하여 이어서 표시 ( 시작점과 끝점을 연결하는 방식)
- PATH LAYER는 이미 이어진 선( 여러 점으로 이어진 ) 데이터를 넣어서 표시
```JS
    new LineLayer({
      id: 'line-layer', // LAYER ID
      data: links, // DATA
      getSourcePosition: d => nodes.find(node => node.name === d.source).coordinates, // 시작점 위경도
      getTargetPosition: d => nodes.find(node => node.name === d.target).coordinates, // 끝점 위경도
      getColor: [255, 255 ,255], // 색
      opacity : 0.4, // 투명도
      auto_highlight: true, // 사용자가 마우스를 올리면 하이라이트 활성화
      highlight_color: [255, 255, 0], // 하이라이트 시 선의 색상
      // picking_radius: 10,
      widthMinPixels: 3, // 선의 최소 두께 설정
    }),


    
    new PathLayer({  
      id: 'lines', // LAYER ID
      data: slinks, // DATA
      getPath: d => d.lines, // 위경도
      getColor: [0, 255 ,255], // 색
      opacity: 0.001, // 투명도
      widthMinPixels: 0.5, // 선의 최소 두께. 최대 두께 : widthMaxPixels
      widthScale: 0.5, // 경로 두께를 조절하는 스케일
      pickable: true, // 경로 상호작용 여부
      rounded: true, // 경로의 끝과 꺾인 부분을 둥글게 렌더링
      /* 최근 아래와 같이 나누어서 사용
      capRounded : true,        
      jointRounded : true
      ============================================ */
      shadowEnabled: false // 그림자 효과 비활성화

    }),

```